In [7]:
!pip install --upgrade --quiet google-genai

In [3]:
from google.colab import userdata

GEMINI_API_KEY : str = userdata.get('GEMINI_API_KEY')

if GEMINI_API_KEY:
  print("GEMINI_API_KEY FOUND")
else:
  print("GEMINI_API_KEY NOT FOUND")

GEMINI_API_KEY FOUND


In [8]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [10]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

In [16]:
magician_video = upload_video('magician.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/hw6znkilkt03


In [17]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = magician_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)


In [18]:
from IPython.display import Markdown
display(Markdown(response.text))

```json
[
  {
    "timecode": "00:00",
    "caption": "A person wearing a blue robe and a purple hat raises their hands above a large black cauldron. Bookshelves line the walls of the wooden room."
  },
  {
    "timecode": "00:01",
     "caption": "Purple smoke rises from the cauldron and swirls around a person wearing a blue robe and a purple hat."
  },
  {
    "timecode": "00:02",
    "caption": "Close-up view of a bubbling purple liquid in a black cauldron, with light beams emerging from within."
  },
  {
    "timecode": "00:03",
    "caption": "A close-up of purple liquid bubbling inside of a black cauldron."
  },
  {
     "timecode":"00:04",
     "caption":"A close up of light beams rising from a bubbling purple liquid in a cauldron."
  }
]
```
